<a href="https://colab.research.google.com/github/myungjinma/Python_Study/blob/main/0819_71%EC%9D%BC%EC%B0%A8_ML(Surprise_%EC%B1%85_%EC%9C%A0%EC%82%AC%EB%8F%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 8.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633973 sha256=27ab58551ccc3cccec9001904194bbf781746144fa6d36c9cc414c1883b14b90
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds

In [5]:
from surprise import Reader

In [3]:
!cp drive/MyDrive/class_data/*.csv ./

In [6]:
import numpy as np
import pandas as pd

In [13]:
file1 = './BX-Book-Ratings.csv'
file2 = './BX_Books.csv'

In [14]:
def pred_data(file1, file2):
  ratings = pd.read_table(file1, sep=';', encoding='latin-1')
  with open('./bbs_edit','w') as g:
    with open(file2, encoding='latin-1') as f:
      line = f.readline()
      while(line):
        real_line = line.replace('\";\"', '###').replace('\"', '')
        g.write(real_line)
        line = f.readline()
  books = pd.read_table('./bbs_edit', sep='###', encoding='latin-1', engine= 'python')
  bookratings = pd.merge(ratings, books, on='ISBN')
  bookratings.rename(columns={'ISBN':'item'}, inplace=True)
  books.rename(columns={'Book-Title':'title', "ISBN":'item'}, inplace=True)
  bookratings.to_csv('./Book-Ratings_noh.csv', index=False, header=False)

In [15]:
pred_data(file1, file2)

In [16]:
!more bbs_edit

ISBN###Book-Title###Book-Author###Year-Of-Publication###Publisher###Image-URL-S#
##Image-URL-M###Image-URL-L
0195153448###Classical Mythology###Mark P. O. Morford###2002###Oxford University
 Press###http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg###http://i
mages.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg###http://images.amazon.com/
images/P/0195153448.01.LZZZZZZZ.jpg
0002005018###Clara Callan###Richard Bruce Wright###2001###HarperFlamingo Canada#
##http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg###http://images.a
mazon.com/images/P/0002005018.01.MZZZZZZZ.jpg###http://images.amazon.com/images/
P/0002005018.01.LZZZZZZZ.jpg
0060973129###Decision in Normandy###Carlo D'Este###1991###HarperPerennial###http
://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg###http://images.amazon.
com/images/P/0060973129.01.MZZZZZZZ.jpg###http://images.amazon.com/images/P/0060
973129.01.LZZZZZZZ.jpg
0374157065###Flu: The Story of the Great Influenza Pandemic of 1918 and th

In [18]:
!head Book-Ratings_noh.csv

276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.LZZZZZZZ.jpg
2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.LZZZZZZZ.jpg
6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.LZZZZZZZ.jpg
8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/03454

In [20]:
def get_data():
  col = 'user item rating'
  reader = Reader(line_format=col, sep=',', rating_scale=(0,10))
  data = Dataset.load_from_file('./Book-Ratings_noh.csv', reader= reader)
  trainset, testset = train_test_split(data, test_size=.25, random_state=0)
  return trainset, testset

In [21]:
trainset, testset = get_data()

In [22]:
trainset

In [ ]:
testset

In [24]:
def learning_test(trainset, testset):
  algo=SVD()
  algo.fit(trainset)
  predictions = algo.test(testset)
  accuracy.rmse(predictions)
  return algo

In [25]:
algo = learning_test(trainset, testset)

RMSE: 3.5104


In [27]:
def full_data_learning():
  col = 'user item rating'
  reader = Reader(line_format=col, sep=',', rating_scale=(0,10))
  data_folds = DatasetAutoFolds(ratings_file='./Book-Ratings_noh.csv', reader = reader)
  trainset = data_folds.build_full_trainset()
  algo = SVD(n_epochs=20, n_factors=50, random_state=0)
  algo.fit(trainset)
  return algo

In [28]:
algo = full_data_learning()

임의의 사용자 uid = 276847이 읽지 않은 책들의 ISBN을 얻어낸다.

In [31]:
def isbn_of_unread(puid):
  data = pd.read_csv('./Book-Ratings_noh.csv', names = ['uid', 'isbn', 'rating', 'title', 'author', 'year','publisher','IS','IM','IL'])
  str_expr = 'uid != @puid'
  df_q = data.query(str_expr)['isbn'].unique()
  return df_q

In [32]:
puid = 276847
unread_ISBN_array = isbn_of_unread(puid)

In [33]:
unread_ISBN_array

array(['034545104X', '0155061224', '0446520802', ..., '0590907301',
       '0679752714', '0806917695'], dtype=object)

In [34]:
unread_ISBN_array.shape

(270162,)

읽지 않은 책에 대해 예측 평점 구하기

In [35]:
def prediction(algo, uid, uia):
  pred = [algo.predict(uid=str(uid), iid = str(isbn)) for isbn in uia[:100]]
  ests = lambda x: x.est
  pred.sort(key=ests, reverse=True)
  top10_pred = pred[:10]
  isbn_est_list = [(pr.iid, pr.est)for pr in top10_pred]
  return isbn_est_list

In [36]:
isbn_est_list = prediction(algo, puid, unread_ISBN_array)

In [37]:
isbn_est_list #276847 이 읽지 않은 책중 평점 높은 순 리스트

[('0060096195', 8.856477559499961),
 ('0440414121', 7.889920872477146),
 ('0440498058', 7.487994845731043),
 ('043935806X', 7.397499186356904),
 ('0439104769', 7.281217315478902),
 ('3442131340', 7.213855518651504),
 ('3548603203', 7.169508155420552),
 ('1562827898', 6.9363309257583206),
 ('3404122879', 6.891915281327378),
 ('0061054151', 6.810686878764974)]

정렬된 ISBN을 이용하여 실제 책 정보 얻어내기

In [38]:
def book_info(prediction):
  books = pd.read_table('./bbs_edit', sep="###", encoding='latin-1', engine='python')
  isbn_list = []
  for i in prediction:
    isbn_list.append(i[0])
  str_expr = 'ISBN in @isbn_list'
  df_q = books.query(str_expr)
  return df_q

In [40]:
df = book_info(isbn_est_list)

In [41]:
print(df)

              ISBN                                         Book-Title  \
3217    3548603203                                      Artemis Fowl.   
5506    043935806X  Harry Potter and the Order of the Phoenix (Boo...   
5731    0440414121  The Watsons Go to Birmingham - 1963 (Yearling ...   
16781   0440498058                                  A Wrinkle In Time   
17084   0060096195                                  The Boy Next Door   
25384   0061054151                            Whirlwind (The X-Files)   
82228   3442131340  Ein Fall fÃ?ÃÂ¼r Kay Scarpetta / Ein Mord fÃ...   
132925  0439104769                                       Bloomability   
173728  3404122879                         Vater Himmel, Mutter Erde.   
246871  1562827898  Storm Surge: A Quin St. James and Mike McClear...   

                    Book-Author  Year-Of-Publication            Publisher  \
3217                Eoin Colfer                 2003      Ullstein TB-Vlg   
5506              J. K. Rowling           

#영화의 장르 유사도 측정 유사도 측정 방법

한 영화는 여러가지 장르를 가질 수 있다.

g_sample= [Action, Fantasy, Fiction]

g_sample을 한 문장으로 바꾼다. :'Action Fantasy Fiction'

sklearn.feature_extraction.test.CountVetorizer 사용

이 형태로 여러 영화의 장르를 한 문장으로 바꾼 데이터를 준비하여 문서처럼 간주한다.

각 개별 장르를 중복없이 나열하고, 각 장르별로 빈도수를 세어 벡터로 만든다.

n_gram_range(1,1) 단어 하나를 토큰화 한다.

n_gram_range(1,2) 단어 하나 뿐만 아니라, 나타난 순서대로 단어 2개씩으로도 토큰화 한다. (1,2) == (범위 최솟값,범위 최댓값)의 의미이다.

In [ ]:
(7 ,3, 1, 4, 1) & (8, 4, 1, 3, 1)= 56 + 12 + 1 + 12 + 1